In [ ]:
# установка mediapy
! pip install mediapy

In [ ]:
# импорт зависимостей
import cv2
import mediapy
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# настройка matplotlib
%config InlineBackend.figure_format = "retina"
plt.style.use("seaborn-v0_8-notebook")

# путь к данным
DATA_DIR = Path("../data")

##### **Детектор границ Собеля**

In [ ]:
f = cv2.imread(
    DATA_DIR / "building.png", 
    cv2.IMREAD_GRAYSCALE
)

sigma = 1.5
n = 2 * np.ceil(3 * sigma).astype(int) + 1
u = cv2.getGaussianKernel(n, sigma=0)
G = u @ u.T

g = cv2.filter2D(f.astype(float), ddepth=-1, kernel=G)

# Ядра Собеля
Sx = 0.25 * np.array([
    [ -1,  0,  1],
    [ -2,  0,  2],
    [-1, 0, 1]
])

Sy = Sx.T

# производные Собеля 
gx = cv2.filter2D(g, ddepth=-1, kernel=Sx)
gy = cv2.filter2D(g, ddepth=-1, kernel=Sy)

# visualization
mediapy.compare_images([gx, gy])

##### *Вычислим модуль градиента*

$$
  m(x,y) =
  \sqrt{
    \left(\frac{\partial g}{\partial x}\right)^2+
    \left(\frac{\partial g}{\partial y}\right)^2
  }
$$

In [ ]:
# модуль градиента
m = np.sqrt((gx**2 + gy**2) / 2)

# visualization
mediapy.compare_images([f, m])

##### *Пороговый постпроцессинг*

$$
  e(x, y) =
  \begin{cases}
      1\,, & m(x,y) > T\\
      0\,, & m(x,y) \leqslant T  
  \end{cases}
$$

In [ ]:
# порог
T = 0.3 * m.max()

# границы Собеля
sobel_edges = m > T

# visualization
mediapy.compare_images([f, sobel_edges])

##### **Детектор границ Кэнни**

In [ ]:
f = cv2.imread(
    DATA_DIR / "building.png", 
    cv2.IMREAD_GRAYSCALE
)

sigma = 1.5
n = 2 * np.ceil(3 * sigma).astype(int) + 1
u = cv2.getGaussianKernel(n, sigma=0)
G = u @ u.T

g = cv2.filter2D(f.astype(float), ddepth=-1, kernel=G)

T1 = 150  
T2 = 50 

canny_edges = cv2.Canny(g.astype(np.uint8), T2, T1)

# visualization
mediapy.compare_images([sobel_edges, canny_edges])

##### **Детектор границ Марра-Хилдрета**

In [ ]:
f = cv2.imread(
    DATA_DIR / "building.png", 
    cv2.IMREAD_GRAYSCALE
)

sigma = 1.5
n = 2 * np.ceil(3 * sigma).astype(int) + 1
u = cv2.getGaussianKernel(n, sigma=0)
G = u @ u.T

# сглаженное изображение
g = cv2.filter2D(f.astype(float), ddepth=-1, kernel=G)

L = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]])

# Лаплас от Гаусса (LoG image)
log_img = cv2.filter2D(g, ddepth=-1, kernel=L)

# visualization
mediapy.compare_images([f, log_img])

In [ ]:
def calc_marr_hildreth_edges(log_img, T):
    H, W = log_img.shape
    
    pad_image = np.zeros((H + 2, W + 2))
    pad_image[1: H + 1, 1: W + 1] = log_img

    edges = np.zeros_like(log_img, dtype=np.uint8)

    for y in range(1, H + 1):
        for x in range(1, W + 1):
            window = pad_image[y - 1: y + 2, x - 1: x + 2]

            (
                a, b, c, 
                d, _, e,  
                f, g, h
            ) = window.flatten()

            ok = False

            if d * e < 0 and np.abs(d - e) > T:
                ok = True
            elif b * g < 0 and np.abs(b - g) > T:
                ok = True
            elif a * h < 0 and np.abs(a - h) > T:
                ok = True
            elif f * c < 0 and np.abs(f - c) > T:
                ok = True

            if ok:
                edges[y - 1, x - 1] = 255

    return edges

In [ ]:
thresh_percent = 0.1 # 10 %
threshold = thresh_percent * log_img.max()

marhil_edges = calc_marr_hildreth_edges(log_img, threshold)

# visualization
mediapy.compare_images([f, marhil_edges])

In [ ]:
# Distance resolution of the accumulator in pixels.
rho_res = 1

# Angle resolution of the accumulator in radians.
theta_red = np.pi / 100

# Accumulator threshold parameter
# Only those lines are returned that get enough votes (> threshold)
threshold = 200

lines = cv2.HoughLines(marhil_edges, rho_res, theta_red, threshold, None, 0, 0)

dst_image = cv2.cvtColor(f, cv2.COLOR_GRAY2RGB)

for i in range(0, len(lines)):
    d = lines[i][0][0]
    theta = lines[i][0][1]

    # нормаль к прямой
    nx = np.cos(theta)
    ny = np.sin(theta)
    
    # origin - точка на прямой 
    x0 = nx * d
    y0 = ny * d
    
    # direction - вектор направления вдоль прямой
    tx = -ny
    ty = nx

    s = 1000

    xy1 = int(x0 + s * tx), int(y0 + s * ty)
    xy2 = int(x0 - s * tx), int(y0 - s * ty)
    
    cv2.line(dst_image, xy1, xy2, (255, 0, 255), 3, cv2.LINE_AA)

# visualization
mediapy.compare_images([f, dst_image])

#### **Домашняя работа 4**

##### **Задача**

<figure>
    <img src="../data/line_ransac.jpg" width="300"/>
    <img src="../data/marr_hildreth_edges.jpg" width="300"/>
</figure>


- найдите границы методом Марра-Хилдрета

- с помощью алгоритма RANSAC, найдите линию на входном изображении

- постройте полученную линию,
а также постройте граничные точки и inliers (множество $\mathbf{S}_i$ в алгоритме RANSAC)

- параметры в файле `params.yaml`:

```yaml
    homework_04:
        # Input image
        input_image: data/building.png
        # Marr-Hildreth edge detection
        sigma: 3.5 
        thresh_percent: 0.2 # T = thresh_percent * LoG.max()
        # RANSAC
        threshold: 5
        max_trials: 5000
        seed: 0xC0FFEE
        # Output images
        output_image: data/line_ransac.jpg
        output_edges: data/marr_hildreth_edges.jpg
```

##### **Теория**

- вычисление градиента изображения с помощью производных Собела, свойства градиента

- детектор границ Собела

- детектор границ Кэнни

- детектор границ Марра-Хилдрета

- детектирование прямых с помощью преобразования Хафа

- алгоритм RANSAC для поиска прямой